In [67]:
import csv
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\seong\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\seong\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\seong\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "c:\Users\seong\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
# Clean the data
with open("player_injuries_impact.csv", "r", encoding="utf8") as stat:
    reader = csv.reader(stat)
    headers = next(reader)  # Read the header row
    playstats = list(range(9)) + [12, 41]  # Columns relevant to the model
    data = []

    for row in reader:
        selected_values = [row[i] if i < len(row) else "" for i in playstats]
        data.append(selected_values)

['Name', 'Team Name', 'Position', 'Age', 'Season', 'FIFA rating', 'Injury', 'Date of Injury', 'Date of return', 'Match1_before_injury_Player_rating', 'Match3_after_injury_Player_rating']


In [ ]:
# Convert data to a NumPy array
data = np.array(data)

Jamaal Lascelles, Newcastle, Center Back, 26, 2019/20, 77, Knee injury, 9-Nov-19, 13-Jan-20, 7.4, 6.7
Fabian Sch?r, Newcastle, Center Back, 28, 2019/20, 79, Knee injury, 20-Oct-19, 24-Nov-19, 5.8, N.A.
Fabian Sch?r, Newcastle, Center Back, 28, 2019/20, 79, Hamstring strain, 2-Jan-20, 17-Jan-20, 4.9, 6.5
Fabian Sch?r, Newcastle, Center Back, 28, 2019/20, 79, Shoulder injury, 16-Jul-20, 28-Sep-20, 5.2, N.A.
Paul Dummett, Newcastle, Center Back, 28, 2019/20, 75, Groin injury, 22-Dec-19, 10-Jan-20, 5.8, N.A.
Ciaran Clark, Newcastle, Center Back, 30, 2019/20, 75, Calf injury, 9-Dec-19, 10-Jan-20, 6, 6.3
Ciaran Clark, Newcastle, Center Back, 30, 2019/20, 75, Ankle injury, 16-Feb-20, 14-Sep-20, 6.3, N.A.
Jetro Willems, Newcastle, Left Back, 26, 2019/20, 77, Knee injury, 22-Sep-19, 5-Oct-19, 6.7, 6.5
Jetro Willems, Newcastle, Left Back, 26, 2019/20, 77, Knee injury, 15-Dec-19, 25-Dec-19, 6, 5.8
Jetro Willems, Newcastle, Left Back, 26, 2019/20, 77, Cruciate ligament tear, 18-Jan-20, 20-Sep-20, 

In [ ]:
# Feature selection
X = data[:, :-1]  # All columns except the last one
y = data[:, -1]   # Last column (target)

In [ ]:
# Handle missing values
X[X == ""] = -1
y[y == ""] = -1

In [ ]:
# Convert strings to numeric
def to_float_safe(value):
    try:
        return float(value)
    except ValueError:
        return -1  # Replace non-numeric values with -1

X = np.vectorize(to_float_safe)(X)
y = np.vectorize(to_float_safe)(y)

In [ ]:
# Target variable y represnets "performance after injury" based on player rating, match performance, etc..
bins = [0, 3, 6, 10]
labels = [0, 1, 2]  
y = np.digitize(y, bins, right=True) - 1  # Assign class labels

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize the RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
# Train the model
rf_model.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)